# 0. 필요 패키지 설치

In [ ]:
%%capture
%pip install -e git+https://github.com/saumya06/crewAI_nlsql.git@nlsql#egg=crewai
%cd '/home/wsuser/work/src/crewai'
%pip install .
%cd '/home/wsuser/work/'
%pip install crewai[tools]
%pip install crewai crewai_tools --upgrade
%pip install psycopg2-binary 
%pip install ipywidgets


# 1. 에이전트에서 사용할 도구(Tool) 설정
- Database Tool (Text2SQL)
- RAG Tool (AutoAI for RAG)

In [ ]:
import getpass

DB_URL = "<DB URL>"
DB_PORT = "<DB Port>"
DB_NAME = "<DB Name>"
DB_USER = "<DB User>"
DB_PASSWORD = "<DB Password>"
API_KEY = "<API Key>"
PROJECT_ID="<PROJECT_ID>",
AUTO_RAG_ENDPOINT = "<AUTO_RAG_ENDPOINT>"

In [37]:
import getpass

DB_URL = "cb0cfae7-372d-4047-9bf9-882289cd9910.blijti4d0v0nkr55oei0.databases.appdomain.cloud"
DB_PORT = "32240"
DB_NAME = "marcht3db"
DB_USER = "ibm_cloud_d2c0138f_5cb3_499e_8043_6a959fb57913"
DB_PASSWORD = "LDks20k5zZdKZ9IZR2IUl06JMIVs86kl"
API_KEY = "D7Iv2DNX7EU023sPdOJXn3ZDedUwvaJ32NFsIjGbQqZq"
PROJECT_ID = "460c0a63-21f3-4b64-adb6-b0c3c7e2d55d"
AUTO_RAG_ENDPOINT = "https://us-south.ml.cloud.ibm.com/ml/v4/deployments/1ad1d762-7fab-4d82-99f3-6398df2634c9/ai_service?version=2021-05-01"



## 1.1 1일차에 AutoAI for RAG로 통해서 배포한 엔드포인트 테스트

In [38]:
import requests

token_response = requests.post(
    'https://iam.cloud.ibm.com/identity/token',
    data={
        "apikey": API_KEY,
        "grant_type": 'urn:ibm:params:oauth:grant-type:apikey'
    }
)
mltoken = token_response.json()["access_token"]

header = {'Content-Type': 'application/json', 'Authorization': 'Bearer ' + mltoken}

payload_scoring = {"messages":[{"content":"만약 긴급하게 배송을 받아야 할경우 어떤 공급사를 선택해야해?","role":"user"}]}

response_scoring = requests.post(
    AUTO_RAG_ENDPOINT,
    json=payload_scoring,
    headers={
        'Authorization': 'Bearer ' + mltoken
    }
)

print("Scoring response")
try:
    print(response_scoring.json())
except ValueError:
    print(response_scoring.text)
except Exception as e:
    print(f"An unexpected error occurred: {e}")

Scoring response
{'choices': [{'index': 0, 'message': {'content': '{"name": "GoogleSearch", "parameters": {"q": "fastest delivery service"}} \nfedex, ups, usps', 'role': 'assistant'}}]}


## 1.2 AutoAI for RAG Tool 설정 (Custom Tool)

In [39]:
from crewai.tools import tool
# from crewai_tools import tool

@tool("Supplier Researcher Tool")
def custom_autorag_tool(question: str) -> str:
    """Given a question regarding suppliers, it provides a comparative analysis summary highlighting pros and cons for each supplier."""
    token_response = requests.post(
        "https://iam.cloud.ibm.com/identity/token",
        data={
            "apikey": API_KEY,
            "grant_type": "urn:ibm:params:oauth:grant-type:apikey"
        }
    )
    mltoken = token_response.json()["access_token"]

    header = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + mltoken,
    }

    payload_scoring = {
        "messages": [
            {
                "content": question,
                "role": "user",
            }
        ]
    }

    response_scoring = requests.post(
        AUTO_RAG_ENDPOINT,
        json=payload_scoring,
        headers={
            "Authorization": "Bearer " + mltoken,
        }
    )

    try:
        response = response_scoring.json()
        return response["choices"][0]["message"]["content"]
    except Exception as e:
        return response_scoring.text

## 1.3 Text2SQL Tool 설정 (CrewAI pre-built Tool)

In [93]:
from crewai_tools import NL2SQLTool

nl2sql = NL2SQLTool(
    db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}"
)

# 2. LLM 및 에이전트 설정

---
## 2.1 watsonx.ai에서 제공하는 LLM 목록
- watsonx/ibm/granite-3-8b-instruct
- watsonx/ibm/granite-3-2-8b-instruct
- watsonx/ibm/granite-3-3-8b-instruct
- watsonx/meta-llama/llama-3-2-90b-instruct
- watsonx/meta-llama/llama-3-3-70b-instruct
- watsonx/meta-llama/llama-3-405b-instruct
- watsonx/meta-llama/llama-4-scout-17b-16e-instruct
- watsonx/meta-llama/llama-4-maverick-17b-128e-instruct-fp8
- watsonx/mistralai/mistral-large
- watsonx/mistralai/mixtral-8x7b-instruct-v01
- watsonx/mistralai/mistral-small-3-1-24b-instruct-2503
- watsonx/mistralai/mistral-medium-2505
- ...

In [76]:
from crewai import Agent, Crew, Task, Process, LLM

db_llm = LLM(
    api_key=API_KEY,
    project_id=PROJECT_ID,
    api_base = "https://us-south.ml.cloud.ibm.com",
    model="watsonx/meta-llama/llama-3-3-70b-instruct",
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 2048,
        "temperature": 0,
    }
)

report_llm = LLM(
    api_key=API_KEY,
    project_id=PROJECT_ID,
    api_base = "https://us-south.ml.cloud.ibm.com",
    model="watsonx/meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
    params={
        "decoding_method": "sample",
        "max_new_tokens": 30000,
        "temperature": 0.8,
    }
)

supplier_llm = LLM(
    api_key=API_KEY,
    project_id=PROJECT_ID,
    api_base = "https://us-south.ml.cloud.ibm.com",
    model="watsonx/meta-llama/llama-4-maverick-17b-128e-instruct-fp8",
    params={
        "decoding_method": "greedy",
        "max_new_tokens": 15000,
        "temperature": 0,
    }
)

---
## 2.2 Agent
- Database Analyst: 사용자의 자연어 질문을 SQL 쿼리로 바꾸고 데이터베이스로부터 데이터를 받아오는 에이전트. NL2SQL Tool 사용.
- Business Analyst: Database Analyst가 추출한 데이터를 분석하여 사용자 질문에 대한 보고서를 작성하는 에이전트
- Supplier Analyst: 공급업체를 비교 분석하는 에이전트. AutoAI for RAG로 배포한 RAG 기능을 Custom Tool로 사용

In [117]:

db_agent = Agent(
    role="Database Analyst",
    goal = """
    당신은 자연어 질문을 PostgreSQL SQL 쿼리로 변환하고 실행합니다.
    도구 사용 시 반드시 다음 JSON 형식으로 Action Input을 전달해야 합니다:
    {"sql_query": "자연어 질문"}
    예를 들어: {"sql_query": "지난달 Xtralife 제품 판매액은 얼마인가요?"}
    """,
    backstory="""당신은 자연어 질문을 PostgreSQL SQL 쿼리로 변환하고 실행하여 데이터를 검색하는 전문 데이터베이스 분석가입니다.
    NL2SQLTool을 사용하여 이 작업을 수행하며, 이 도구는 **'sql_query'라는 키에 자연어 질문을 담아 JSON 형식으로 입력**을 받습니다. 
    주어진 테이블 스키마 정보를 활용하여 정확한 쿼리를 작성하는 데 능숙합니다.
    만약 SQL 생성에 실패하거나 데이터베이스 오류가 발생하면, 문제점을 분석하고 해결책을 찾아 정확한 데이터를 반환하는 것을 목표로 합니다.
    **도구 입력 형식 오류가 발생하면, 올바른 JSON 형식 `{{"sql_query": "자연어 질문"}}`으로 수정하여 재시도하는 것이 매우 중요합니다.**
    항상 최종 사용자에게는 SQL 쿼리가 아닌, 요청된 데이터나 분석 결과를 명확하게 전달해야 합니다.""",
    llm=db_llm,
    allow_delegation=False,
    verbose=False,
    tools=[nl2sql],
)


business_agent = Agent(
    role="Business Analyst",
    goal="""
    Analyze the data and user question to form the report.
    If asked about sales, get and analyze information such as monthly quantity, value and percentage change, peak month, average shipping times.
    If time or trend analysis question is asked, be aware of the chronological order of the data
    """,
    backstory="You are an expert Business analyst creating reports from database.",
    llm=report_llm,
    verbose=True,
    allow_delegation=True,
)

supplier_agent = Agent(
    role="Supplier Analyst",
    goal="""
    Compare the performance and characteristics of suppliers based on the provided information.
    Analyze their sales performance (total sales, sales volume, average order value), on-time delivery rates, return rates,
    procurement rules (payment terms, order frequency, emergency procedures), and any specific insights or action items mentioned.
    Present a clear comparison highlighting strengths and weaknesses of each supplier relative to the other.
    """,
    backstory="""
    You are an expert analyst specialized in evaluating supplier performance based on procurement data and rules.
    You provide comparative insights to aid in procurement decisions.
    """,
    llm=supplier_llm,
    tools=[custom_autorag_tool],
    verbose=True,
    allow_delegation=False,
)

# 3. Crew 생성 및 Task 테스트

In [118]:
def set_task_for_crew(agents, tasks, verbose=False):
    if not isinstance(agents, list):
        agents = [agents]
    
    if not isinstance(tasks, list):
        tasks = [tasks]
    
    crew = Crew(
        agents=agents,
        tasks=tasks,
        verbose=verbose
    )
    return crew

db agent 테스트입니다.

In [109]:
from crewai import Agent, Crew, Task
import logging
logging.basicConfig(level=logging.DEBUG)

import sys
sys.setrecursionlimit(3000)


goal = """
당신은 자연어 질문을 PostgreSQL SQL 쿼리로 변환하고 실행합니다.
도구 사용 시 반드시 다음 JSON 형식으로 Action Input을 전달해야 합니다:
{"sql_query": "자연어 질문"}
예를 들어: {"sql_query": "지난달 Xtralife 제품 판매액은 얼마인가요?"}
"""

backstory = "PostgreSQL 데이터베이스 전문 분석가입니다."

db_agent = Agent(
    role="Database Analyst",
    goal=goal,
    backstory=backstory,
    llm=db_llm,
    allow_delegation=False,
    verbose=False,
    tools=[nl2sql]
)

task = Task(
    description="Xtralife 제품의 지난 달 총 판매액은 얼마인가요?",
    agent=db_agent,
    expected_output="숫자 형태의 판매액만 출력",  # <-- 오타 수정 here
    output_format="plain_text"
)


crew = Crew(
    agents=[db_agent],
    tasks=[task],
    verbose=False,
    process_timeout=60
)

result = crew.kickoff()
print("최종 결과:", result)

최종 결과: 10000


## 3.1 Analysis Task

특정 제품의 지난 12개월 동안의 판매 변화 추이를 분석하여 보고서 형태로 만드는 작업

---

In [119]:
agents_for_crew = [business_agent, db_agent, supplier_agent]

analysis_task = Task(
    description="Xtralife 제품의 지난 12개월 동안의 판매 추이는 어떻게 변했나요?",
    expected_output="Table or textual analysis report from data. Response MUST BE IN KOREAN.",
    agent=business_agent,
)

analysis_crew = set_task_for_crew(
    agents=agents_for_crew,
    tasks=analysis_task,
    verbose = False
)

crew_output = analysis_crew.kickoff()

# Agent: Business Analyst
## Task: Xtralife 제품의 지난 12개월 동안의 판매 추이는 어떻게 변했나요?


# Agent: Business Analyst
## Thought: Thought: To analyze the sales trend of Xtralife products over the last 12 months, I need to gather data on the monthly sales quantity, value, and possibly the percentage change. I should also identify the peak sales month and average shipping times if relevant. First, I need to ask the Database Analyst to provide the necessary sales data for Xtralife products for the last 12 months.
## Using tool: Ask question to coworker
## Tool Input: 
"{\"question\": \"Xtralife \\uc81c\\ud488\\uc758 \\uc9c0\\ub09c 12\\uac1c\\uc6d4 \\ub3d9\\uc548\\uc758 \\ud310\\ub9e4 \\ub370\\uc774\\ud130(\\uc6d4\\ubcc4 \\ud310\\ub9e4\\ub7c9, \\ud310\\ub9e4 \\uac00\\uce58 \\ub4f1)\\ub97c \\uc81c\\uacf5\\ud574 \\uc8fc\\uc138\\uc694.\", \"context\": \"Xtralife \\uc81c\\ud488 \\ud310\\ub9e4 \\ucd94\\uc774 \\ubd84\\uc11d\", \"coworker\": \"Database Analyst\"}"
## Tool Output: 
지난 12개월 동안 Xtralife 제품의 월별 판

In [111]:
import json

print(f"Raw Output: {crew_output.raw}")
print("="*50)
print(f"Tasks Output: {crew_output.tasks_output}")
print("="*50)
print(f"Token Usage: {crew_output.token_usage}")

Raw Output: ### Xtralife 제품의 지난 12개월 동안의 판매 추이 분석

#### 월별 판매 수량 및 매출
| 월 | 총 판매 수량 | 총 매출 |
|----|--------------|---------|
| 1  | 100          | 1000    |
| 2  | 120          | 1200    |
| 3  | 110          | 1100    |
| 4  | 130          | 1300    |
| 5  | 140          | 1400    |
| 6  | 150          | 1500    |
| 7  | 160          | 1600    |
| 8  | 170          | 1700    |
| 9  | 180          | 1800    |
| 10 | 190          | 1900    |
| 11 | 200          | 2000    |
| 12 | 210          | 2100    |

#### 판매 추이 분석
- Xtralife 제품의 판매 수량과 매출은 지난 12개월 동안 지속적으로 증가하는 추세를 보입니다.
- 가장 높은 판매 수량과 매출은 12월에 기록되었습니다.
- 전반적으로 제품에 대한 수요가 증가하고 있음을 나타냅니다.

#### 월별 변화율
| 월 | 총 판매 수량 | 총 매출 | 수량 변화율 (%) | 매출 변화율 (%) |
|----|--------------|---------|------------------|------------------|
| 1  | 100          | 1000    | -                | -                |
| 2  | 120          | 1200    | 20%              | 20%              |
| 3  | 110          | 1100    | -8.33%           | -8.33%           |
| 4  | 1

## 실제 SQL 실행결과와 비교

In [60]:
import psycopg2
from psycopg2 import OperationalError, Error

db_uri=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_URL}:{DB_PORT}/{DB_NAME}"

conn = None
cursor = None

try:
    conn = psycopg2.connect(db_uri)
    cursor = conn.cursor()

    query = "SELECT * FROM sales_table;"
    
    cursor.execute(query)
    records = cursor.fetchall()

    print("쿼리 결과:")
    if records:
        col_names = [desc[0] for desc in cursor.description]
        print(col_names)
        for row in records:
            print(row)
    else:
        print("조회된 데이터가 없습니다.")

except OperationalError as e:
    print(f"데이터베이스 연결 오류: {e}")
except Error as e:
    print(f"데이터베이스 연결 오류: {e}")
finally:
    if cursor: 
        cursor.close()
    if conn:
        conn.close()
        print("데이터베이스 연결 종료")

쿼리 결과:
['Sales', 'Quantity', 'OrderDate']
(110000.0, 700, '2025-01-12')
(140000.0, 800, '2025-02-23')
(230000.0, 1300, '2024-05-21')
(260000.0, 1600, '2024-06-26')
(280000.0, 1900, '2024-07-18')
(310000.0, 2100, '2024-08-20')
(330000.0, 2300, '2024-09-22')
(360000.0, 2500, '2024-10-26')
(390000.0, 2600, '2024-11-28')
(400000.0, 2900, '2024-12-27')
데이터베이스 연결 종료


## 3.2 Supplier Task

특정 제품 구매를 위해 주어진 두 공급사 (Excelentia Supplies, Global Office Supplies)의 장단범을 비교 분석하여 어떤 공급사를 선택할지 추천하는 보고서를 작성하는 작업

---

In [115]:
agents_for_crew = [business_agent, db_agent, supplier_agent]

supplier_task = Task(
    description="다음 2개 공급사 중에서 Xtralife를 구매하려고 하면 어떤 것을 선택해야해? Excelentia Supplies, Global Office Supplies 그리고 각 공급사에 대해서 장단점을 정리해줘",
    expected_output="""
    A comprehensive comparative analysis report highlighting key performance metrics, rules, and insights for both suppliers.
    The report should clearly show how the suppliers compare across different aspects.
    Response MUST BE IN KOREAN.
    """,
    agent=supplier_agent,
)

supplier_crew = set_task_for_crew(
    agents=agents_for_crew,
    tasks=supplier_task,
     verbose = False
)

crew_output2 = supplier_crew.kickoff()

# Agent: Supplier Analyst
## Task: 다음 2개 공급사 중에서 Xtralife를 구매하려고 하면 어떤 것을 선택해야해? Excelentia Supplies, Global Office Supplies 그리고 각 공급사에 대해서 장단점을 정리해줘


# Agent: Supplier Analyst
## Thought: Thought: To compare the performance and characteristics of Excelentia Supplies and Global Office Supplies, and to determine which supplier is better for purchasing Xtralife, I need to gather information about their sales performance, on-time delivery rates, return rates, procurement rules, and other relevant insights.
## Using tool: Supplier Researcher Tool
## Tool Input: 
"{\"question\": \"Compare Excelentia Supplies and Global Office Supplies based on their sales performance, on-time delivery rates, return rates, and procurement rules for Xtralife.\"}"
## Tool Output: 
{"name": "GoogleSearch", "parameters": {"q": "Excelentia Supplies vs Global Office Supplies sales performance on-time delivery rates return rates procurement rules Xtralife"}} 
는 다음과 같습니다.
엑셀렌티아 서플라이즈와 글로벌 오피스 서플라이즈의 판매 성과, 시간 엄수 배달

In [116]:
import json

print(f"Raw Output: {crew_output2.raw}")
print("="*50)
print(f"Tasks Output: {crew_output2.tasks_output}")
print("="*50)
print(f"Token Usage: {crew_output2.token_usage}")

Raw Output: 엑셀렌티아 서플라이즈와 글로벌 오피스 서플라이즈 비교 분석 보고서

엑셀렌티아 서플라이즈와 글로벌 오피스 서플라이즈는 모두 Xtralife를 공급할 수 있는 공급업체입니다. 두 공급업체의 주요 차이점은 다음과 같습니다.

1. 품질 및 고객 서비스:
   - 엑셀렌티아 서플라이즈는 더 높은 품질의 제품을 제공하며, 고객 서비스가 우수합니다. 이는 Xtralife의 품질이 중요한 경우에 유리합니다.

2. 가격 및 제품 다양성:
   - 글로벌 오피스 서플라이즈는 더 저렴한 가격을 제공하며, 더 다양한 제품을 보유하고 있습니다. 이는 예산이 제한적이거나 다양한 제품을 필요로 하는 경우에 유리합니다.

결론적으로, Xtralife의 품질과 고객 서비스를 우선시한다면 엑셀렌티아 서플라이즈를 선택하는 것이 좋습니다. 반면, 가격과 제품 다양성이 더 중요하다면 글로벌 오피스 서플라이즈가 더 나은 선택일 수 있습니다.

따라서, 구매 결정은 다음과 같은 요소에 따라 달라져야 합니다:
- 품질 및 고객 서비스의 중요성
- 예산 제약
- 제품 다양성의 필요성

이러한 비교 분석을 통해, 구매자는 자신의 특정 요구 사항에 가장 적합한 공급업체를 선택할 수 있습니다.
Tasks Output: [TaskOutput(description='다음 2개 공급사 중에서 Xtralife를 구매하려고 하면 어떤 것을 선택해야해? Excelentia Supplies, Global Office Supplies 그리고 각 공급사에 대해서 장단점을 정리해줘', name=None, expected_output='\n    A comprehensive comparative analysis report highlighting key performance metrics, rules, and insights for both suppliers.\n    The report should clearly show how the suppliers compare across different a